In [1]:
import sys
import os
sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())))

In [2]:
import copy
import torch
from datetime import datetime
from utils.helper import ModelConfig, color_print
from utils.dataset_utils.load_dataset import (
    load_data,
    convert_dataset_labels_to_binary,
)
from utils.model_utils.load_model import load_model
from utils.model_utils.evaluate import evaluate_model, get_sparsity
from utils.model_utils.save_module import save_module
from utils.decompose_utils.weight_remover import WeightRemoverBert
from utils.decompose_utils.concern_identification import ConcernIdentificationBert
from utils.decompose_utils.tangling_identification import TanglingIdentification
from utils.decompose_utils.concern_modularization import ConcernModularizationBert
from utils.decompose_utils.sampling import sampling_class
from utils.prune_utils.prune import prune_magnitude, find_layers, LayerWrapper, prune_ci, prune_concern_identification

In [3]:
name = "OSDG"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

checkpoint = None
model_config = ModelConfig(name, device)
num_labels = model_config.config["num_labels"]

In [4]:
i = 0
model, tokenizer, checkpoint = load_model(model_config)

train_dataloader, valid_dataloader, test_dataloader = load_data(name, batch_size=64)

color_print("Start Time:" + datetime.now().strftime("%H:%M:%S"))
color_print("#Module " + str(i) + " in progress....")
num_samples = 64

positive_samples = sampling_class(
    train_dataloader, i, num_samples, num_labels, True, 4, device=device
)
negative_samples = sampling_class(
    train_dataloader, i, num_samples, num_labels, False, 4, device=device
)

all_samples = sampling_class(
    train_dataloader, 200, 20, num_labels, False, 4, device=device
)

print("origin")
# evaluate_model(model, model_config, test_dataloader)

module = copy.deepcopy(model)
wr = WeightRemoverBert(model, p=0.9)
ci = ConcernIdentificationBert(model, p=0.4)
ti = TanglingIdentification(model, p=0.5)

print("Start Magnitude pruning")
prune_magnitude(module, include_layers=["attention", "intermediate", "output"], sparsity_ratio=0.1)
print(get_sparsity(module)[0])
print("Start Positive CI after sparse")

prune_concern_identification(
    model,
    module,
    positive_samples,
    include_layers=["attention", "intermediate", "output"],
    sparsity_ratio=0.6,
)

print(get_sparsity(module))
# result = evaluate_model(module, model_config, test_dataloader)
torch.cuda.empty_cache()

Loading the model.
{'model_name': 'sadickam/sdg-classification-bert', 'task_type': 'classification', 'architectures': 'bert', 'dataset_name': 'OSDG', 'num_labels': 16, 'cache_dir': 'Models'}
The model sadickam/sdg-classification-bert is loaded.
{'dataset_name': 'OSDG', 'path': 'albertmartinez/OSDG', 'config_name': '2024-01-01', 'text_column': 'text', 'label_column': 'labels', 'cache_dir': 'Datasets/OSDG', 'task_type': 'classification'}
Loading cached dataset OSDG.
The dataset OSDG is loaded
Start Time:16:38:12
#Module 0 in progress....
origin
Start Magnitude pruning
0.09919858441371328
Start Positive CI after sparse
(0.5944862664659172, {'bert.encoder.layer.0.attention.self.query.weight': 0.5989583333333334, 'bert.encoder.layer.0.attention.self.query.bias': 0.0, 'bert.encoder.layer.0.attention.self.key.weight': 0.5989583333333334, 'bert.encoder.layer.0.attention.self.key.bias': 0.0, 'bert.encoder.layer.0.attention.self.value.weight': 0.5989634195963541, 'bert.encoder.layer.0.attention.

In [5]:
from utils.prune_utils.prune import recover_tangling_identification

In [6]:
recover_tangling_identification(model,
    module,
    negative_samples,
    sparsity_ratio=0.1,
    include_layers=["attention", "intermediate", "output"])

In [7]:
print(get_sparsity(module))

(0.49528768205220397, {'bert.encoder.layer.0.attention.self.query.weight': 0.4989590115017361, 'bert.encoder.layer.0.attention.self.query.bias': 0.0, 'bert.encoder.layer.0.attention.self.key.weight': 0.4989590115017361, 'bert.encoder.layer.0.attention.self.key.bias': 0.0, 'bert.encoder.layer.0.attention.self.value.weight': 0.4989640977647569, 'bert.encoder.layer.0.attention.self.value.bias': 0.0, 'bert.encoder.layer.0.attention.output.dense.weight': 0.4989590115017361, 'bert.encoder.layer.0.attention.output.dense.bias': 0.0, 'bert.encoder.layer.0.intermediate.dense.weight': 0.4999351501464844, 'bert.encoder.layer.0.intermediate.dense.bias': 0.0, 'bert.encoder.layer.0.output.dense.weight': 0.4989585876464844, 'bert.encoder.layer.0.output.dense.bias': 0.0, 'bert.encoder.layer.1.attention.self.query.weight': 0.4989590115017361, 'bert.encoder.layer.1.attention.self.query.bias': 0.0, 'bert.encoder.layer.1.attention.self.key.weight': 0.4989590115017361, 'bert.encoder.layer.1.attention.self.k

In [8]:
result = evaluate_model(module, model_config, test_dataloader)

Evaluating: 100%|██████████| 200/200 [02:49<00:00,  1.18it/s]


Loss: 1.7900
Precision: 0.7052, Recall: 0.4382, F1-Score: 0.4729
              precision    recall  f1-score   support

           0       0.75      0.36      0.48       797
           1       0.77      0.19      0.30       775
           2       0.91      0.50      0.64       795
           3       0.94      0.35      0.51      1110
           4       0.23      0.90      0.37      1260
           5       0.91      0.10      0.18       882
           6       0.86      0.36      0.51       940
           7       0.42      0.14      0.21       473
           8       0.59      0.52      0.55       746
           9       0.38      0.61      0.47       689
          10       0.83      0.33      0.47       670
          11       0.75      0.21      0.32       312
          12       0.64      0.54      0.59       665
          13       0.85      0.43      0.57       314
          14       0.83      0.52      0.64       756
          15       0.62      0.96      0.75      1607

    accuracy   